In [1]:
import os
import argparse
from time import time
from pathlib import Path
import pandas as pd     

In [5]:
 df = pd.read_csv('../data/mta_turnstile_230318.csv.gz', iterator=False,compression="gzip")
 df.head(10)

,Unnamed: 0,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/11/2023,03:00:00,REGULAR,7841834,2793522
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/11/2023,07:00:00,REGULAR,7841838,2793529
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/11/2023,11:00:00,REGULAR,7841864,2793593
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/11/2023,15:00:00,REGULAR,7841929,2793634
4,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/11/2023,19:00:00,REGULAR,7842065,2793683
5,5,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/11/2023,23:00:00,REGULAR,7842124,2793709
6,6,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2023,04:00:00,REGULAR,7842135,2793724
7,7,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2023,08:00:00,REGULAR,7842141,2793742
8,8,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2023,12:00:00,REGULAR,7842174,2793772
9,9,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/12/2023,16:00:00,REGULAR,7842238,2793823


In [22]:
# add the datetime column and drop the DATE and TIME columns
df['CREATED'] =  pd.to_datetime(df['DATE'] + ' ' + df['TIME'], format='%m/%d/%Y %H:%M:%S')
df = df.drop('DATE', axis=1).drop('TIME',axis=1)
df.head(10)

,Unnamed: 0,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,ENTRIES,EXITS,CREATED
0,0,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7841834,2793522,2023-03-11 03:00:00
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7841838,2793529,2023-03-11 07:00:00
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7841864,2793593,2023-03-11 11:00:00
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7841929,2793634,2023-03-11 15:00:00
4,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7842065,2793683,2023-03-11 19:00:00
5,5,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7842124,2793709,2023-03-11 23:00:00
6,6,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7842135,2793724,2023-03-12 04:00:00
7,7,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7842141,2793742,2023-03-12 08:00:00
8,8,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7842174,2793772,2023-03-12 12:00:00
9,9,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7842238,2793823,2023-03-12 16:00:00
